In [5]:
import torch

[-]E1009 00:31:55.662383 251867 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_15sgt1htgWq7[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>


In [6]:
import os
import sys
import logging
from monarch.actor import ProcMesh, Actor, endpoint, current_rank
import socket
from torchtitan.tools.logging import init_logger, logger
from torchtitan.train import Trainer
from typing import Optional
import torch
from torchtitan.config import JobConfig


class TitanTrainerWrapper(Actor):
    def __init__(self, job_config: JobConfig):
        self.rank = current_rank().rank
        self.job_config = job_config

    def _rprint(self, msg):
        """Helper method to print with rank information."""
        print(f"{self.rank=} {msg}")

    @endpoint
    def init(self):
        logging.getLogger().addHandler(logging.StreamHandler(sys.stderr))
        print(f"Initializing actor: {self.rank} {current_rank()=} {socket.gethostname()=}")


    @endpoint
    def train(self):
        logger.info("Starting training")
        config = self.job_config
        trainer: Optional[Trainer] = None

        try:
            trainer = Trainer(config)
            trainer.train()

            if config.checkpoint.create_seed_checkpoint:
                assert (
                    int(os.environ["WORLD_SIZE"]) == 1
                ), "Must create seed checkpoint using a single device, to disable sharding."
                assert (
                    # config.checkpoint.enable_checkpoint
                    config.checkpoint.enable
                ), "Must enable checkpointing when creating a seed checkpoint."
                trainer.checkpointer.save(curr_step=0, )
                logger.info("Created seed checkpoint")
            else:
                trainer.train()
        finally:
            if trainer:
                trainer.close()

            if torch.distributed.is_initialized():
                torch.distributed.destroy_process_group()
                logger.info("Process group destroyed.")
        print("Done training")

[-]E1009 00:32:11.680823 251867 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_15sgt1htgWq7[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>


In [7]:
%cd /mnt/models/mreso/monarch/examples/

/mnt/models/mreso/monarch/examples


[-]E1009 00:32:15.684525 251867 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_15sgt1htgWq7[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>


In [ ]:
from torchtitan.config import ConfigManager, JobConfig

init_logger()
config_manager = ConfigManager()

job_name = "MYJOB"


manual_args = [
        "--job.config_file",
        # os.path.expanduser("/mnt/models/mreso/torchtitan/torchtitan/models/llama3/train_configs/debug_model.toml"),
        os.path.expanduser("/mnt/models/mreso/torchtitan/torchtitan/models/llama3/train_configs/llama3_8b.toml"),
        "--model.tokenizer-path",
        "/mnt/models/mreso/torchtitan/assets/hf/Llama-3.1-8B",
        "--training.steps",
        "25",
        # "--training.dataset_path",
        # "/mnt/models/mreso/torchtitan/tests/assets/c4_test/",
        # # f"{FUSE_DST}/c4",
        # "/teamspace/studios/this_studio/torchtitan/tests/assets/c4_test",
        "--job.dump_folder",
        # f"{FUSE_DST}/outputs/" + job_name,
        "/mnt/models/mreso/torchtitan/outputs/" + job_name
    ]
config = config_manager.parse_args(manual_args)


# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

# @noautodeps
# pyre-ignore-all-errors
import logging
import os
import os
import torch
import torch.distributed as dist
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim

from monarch.tools import commands
from monarch.actor import Actor, current_rank, endpoint
from monarch.actor import Actor, current_rank, endpoint
from monarch.utils import setup_env_for_distributed
from torch.nn.parallel import DistributedDataParallel as DDP
from slurm.utils import get_appdef, get_server_info, create_proc_mesh


logging.basicConfig(
    level=logging.INFO,
    format="%(name)s %(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)


logger: logging.Logger = logging.getLogger(__name__)


async def main(job_config: JobConfig):
    num_hosts = 2
    appdef = await get_appdef(num_hosts)

    appdef.roles[0].resource.gpu = 8

    server_info = await get_server_info(appdef)

    try:
        proc_mesh = await create_proc_mesh(num_hosts, appdef, server_info)
        
        await setup_env_for_distributed(proc_mesh)
        
        await proc_mesh.logging_option(stream_to_client=True, aggregate_window_sec=None)

        print(job_config)

        trainer_actor = await proc_mesh.spawn("trainer_actor", TitanTrainerWrapper, job_config)
        await trainer_actor.init.call()
        await trainer_actor.train.call()

        print("Titan example completed successfully!")

    finally:
        commands.kill(f"slurm:///{server_info.name}")



await main(config)

[titan] 2025-10-09 00:32:18,747 - root - WARNING - tokenizer_path is deprecated, use model.hf_assets_path instead. Setting hf_assets_path to tokenizer_path temporarily.


<string>:8: FutureWarning: Setting `workspace='/mnt/models/mreso/monarch/examples'` is deprecated. Use `workspace=monarch.tools.config.workspace.Workspace(dirs=['/mnt/models/mreso/monarch/examples'])` instead.
torchx.schedulers.slurm_scheduler 2025-10-09 00:32:18 INFO unable to get job info for `monarch-root` with `squeue` (squeue: error: Invalid job id: monarch-root
), trying `sacct`
torchx.schedulers.slurm_scheduler 2025-10-09 00:32:18 INFO unable to get job info for `monarch-root` with `sacct` (sacct: fatal: Bad job/step specified: monarch-root
)
monarch.tools.commands 2025-10-09 00:32:18 INFO no existing RUNNING server `slurm:///monarch-root` creating new one...


torchx.runner.api 2025-10-09 00:32:18 INFO Tracker configurations: {}
torchx.runner.api 2025-10-09 00:32:18 INFO Checking for changes in workspace `/root/.monarch/out/tmp4qntx30d/workspace`...
torchx.runner.api 2025-10-09 00:32:18 INFO To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.
torchx.runner.api 2025-10-09 00:32:18 INFO Reusing original image `monarch_default_workspace:latest` for role[0]=mesh0. Either a patch was built or no changes to workspace was detected.
monarch.tools.commands 2025-10-09 00:32:18 INFO created new `slurm:///1672` waiting for it to be ready...


app_id='monarch-root'
slurm_24_11_0=(24, 11)
slurm_version=(21, 8)
script='#!/bin/bash\n#\n# Generated by TorchX 0.8.0dev0\n# Run with: sbatch --parsable /tmp/tmpd8bvzgie/torchx-sbatch.sh\n#\n#SBATCH --job-name=mesh0-0 --requeue --ntasks-per-node=1 --cpus-per-task=48 --mem=186777 --gpus-per-node=8\n\nset -evx\n\nexport PYTHONUNBUFFERED=1\nexport SLURM_UNBUFFEREDIO=1\nexport TORCHX_MAX_RETRIES=0\n\nset +e\nsrun --output=slurm-"$SLURM_JOB_ID"-mesh0-0.out --error=slurm-"$SLURM_JOB_ID"-mesh0-0.err --wait=60 --kill-on-bad-exit=1 --export=ALL,WORKSPACE_DIR=monarch_default_workspace:latest/workspace,PYTHONPATH=monarch_default_workspace:latest/workspace/,CONDA_DIR=monarch_default_workspace:latest/conda,TORCHX_JOB_ID=slurm:///"$SLURM_JOB_ID",TORCHX_INTERNAL_SESSION_ID=11dc95d9-17cb-49a0-bbea-a2e4af246e00 process_allocator --port=26600 --program=monarch_bootstrap\nexitcode=$?\nset -e\n\necho "job exited with code $exitcode"\nif [ $exitcode -ne 0 ]; then\n    if [ "$TORCHX_MAX_RETRIES" -gt "${SLU

slurm.utils 2025-10-09 00:32:23 INFO 
===== Server Info =====
{
  "name": "1672",
  "server_handle": "slurm:///1672",
  "state": "RUNNING",
  "meshes": {
    "mesh0": {
      "host_type": "__UNSET__",
      "hosts": 1,
      "gpus": -1,
      "hostnames": [
        "chi2599"
      ]
    }
  }
}
monarch._src.actor.allocator 2025-10-09 00:32:23 INFO no match label `procmesh.monarch.meta.com/name` specified in alloc constraints
monarch._src.actor.allocator 2025-10-09 00:32:23 INFO found a single proc mesh `mesh0` in slurm:///1672, will allocate on it
monarch.tools.network 2025-10-09 00:32:23 INFO no AF_INET6 address that can bind TCP sockets for `chi2599:26600` (error: [Errno -3] Temporary failure in name resolution)
monarch.tools.network 2025-10-09 00:32:23 INFO resolved AF_INET address `108.61.203.106:26600` for `chi2599:26600`
monarch._src.actor.allocator 2025-10-09 00:32:23 INFO initializing alloc on remote allocator addresses: ['tcp!108.61.203.106:26600']
monarch._src.actor.allocator

...

[chi2599 2059106] Starting training
[chi2599 2059106] Starting job: Llama 3 8B training
[chi2599 2059106] Building 1-D device mesh with ['dp_shard'], [8]
[chi2599 2059106] [GC] Initial GC collection took 0.00 seconds
[chi2599 2059104] Starting training
[chi2599 2059103] Starting training
[chi2599 2059101] Starting training
[chi2599 2059104] Starting job: Llama 3 8B training
[chi2599 2059102] Starting training
[chi2599 2059104] Building 1-D device mesh with ['dp_shard'], [8]
[chi2599 2059103] Starting job: Llama 3 8B training
[chi2599 2059101] Starting job: Llama 3 8B training
[chi2599 2059102] Starting job: Llama 3 8B training
[chi2599 2059105] Starting training
[chi2599 2059103] Building 1-D device mesh with ['dp_shard'], [8]
[chi2599 2059101] Building 1-D device mesh with ['dp_shard'], [8]
[chi2599 2059104] [GC] Initial GC collection took 0.00 seconds
[chi2599 2059105] Starting job: Llama 3 8B training
[chi2599 2059102] Building 1-D device mesh with ['dp_shard'], [8]
[chi2599 2059107

[chi2599 2059101] Done training
[chi2599 2059104] Done training
[chi2599 2059107] Done training
[chi2599 2059106] Done training
[chi2599 2059105] Done training
[chi2599 2059102] Done training
[chi2599 2059103] Done training


[chi2599 2059101] Process group destroyed.
[chi2599 2059104] Process group destroyed.
[chi2599 2059107] Process group destroyed.
[chi2599 2059106] Process group destroyed.
[chi2599 2059105] Process group destroyed.
[chi2599 2059103] Process group destroyed.
[chi2599 2059102] Process group destroyed.
[chi2599 2059100] Training completed
[chi2599 2059100] Training starts at step 26
[chi2599 2059100] Profiling active. Traces will be saved at /mnt/models/mreso/torchtitan/outputs/MYJOB/profile_trace
[chi2599 2059100] Sleeping 2 seconds for other ranks to complete
[chi2599 2059100] Training completed


[chi2599 2059100] Done training
Titan example completed successfully!
app_id='1672'
output='{\n  "meta": {\n    "plugin": {\n      "type": "openapi\\/v0.0.37",\n      "name": "Slurm OpenAPI v0.0.37"\n    },\n    "Slurm": {\n      "version": {\n        "major": 21,\n        "micro": 5,\n        "minor": 8\n      },\n      "release": "21.08.5"\n    }\n  },\n  "errors": [\n  ],\n  "jobs": [\n    {\n      "account": "",\n      "accrue_time": 1759969938,\n      "admin_comment": "",\n      "array_job_id": 0,\n      "array_task_id": null,\n      "array_max_tasks": 0,\n      "array_task_string": "",\n      "association_id": 0,\n      "batch_features": "",\n      "batch_flag": true,\n      "batch_host": "chi2599",\n      "flags": [\n        "JOB_CPUS_SET ",\n        "JOB_WAS_RUNNING",\n        "JOB_MEM_SET"\n      ],\n      "burst_buffer": "",\n      "burst_buffer_state": "",\n      "cluster": "vultr-mi325x-torch",\n      "cluster_features": "",\n      "command": "\\/tmp\\/tmpd8bvzgie\\/torchx-

[chi2599 2059100] Process group destroyed.
[-]E1009 00:33:33.780559 251867 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_15sgt1htgWq7[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>


In [9]:
proc_mesh.stop()

NameError: name 'proc_mesh' is not defined

[-]E1009 00:33:41.791725 251867 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_15sgt1htgWq7[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>
